In [1]:
import os
os.environ['OMP_NUM_THREADS'] = '6'  # Adjust the number of threads as necessary

In [2]:
import networkx as nx
import numpy as np
import csv

def build_multidigraph_from_csv(csv_file):
    G = nx.MultiDiGraph()

    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            # Exclude 'no_relation' edges
            if row['relation_type'] != 'no_relation':
                # Add nodes with the 'name' attribute
                G.add_node(row['starter_ID'], name=row['starter_ID'])
                G.add_node(row['receiver_ID'], name=row['receiver_ID'])

                # Add directed edges with additional attributes
                weight = float(row['weight'])
                G.add_edge(
                    row['starter_ID'], 
                    row['receiver_ID'], 
                    weight=weight,
                    interaction_type=row['subtype_name'],
                    relation_type=row['relation_type'],
                    pathway_sources=row['pathway_source'],
                    credibility=row['credibility']
                )
    
    return G

def create_global_node_to_index_mapping(train_graph, val_graph):
    all_nodes = set(train_graph.nodes()).union(set(val_graph.nodes()))
    return {node: i for i, node in enumerate(all_nodes)}

# Paths to the CSV files
train_csv_path = 'relations_train_final.csv'
val_csv_path = 'cleaned_relations_val_final.csv'
# Build the MultiDiGraphs
train_MDG = build_multidigraph_from_csv(train_csv_path)
val_MDG = build_multidigraph_from_csv(val_csv_path)

# Create a global node to index mapping
global_node_to_index = create_global_node_to_index_mapping(train_MDG, val_MDG)

In [3]:
import torch
from torch_geometric.utils import from_networkx

def apply_mapping_and_get_indices(graph, mapping):
    # Create a tensor of node indices based on the global mapping
    num_nodes = len(mapping)
    node_indices = torch.arange(num_nodes)

    # Remap nodes in the graph according to the global mapping
    remapped_graph = nx.relabel_nodes(graph, mapping)

    return remapped_graph, node_indices

# Apply mapping to training and validation graphs
remapped_train_MDG, train_indices = apply_mapping_and_get_indices(train_MDG, global_node_to_index)
remapped_val_MDG, val_indices = apply_mapping_and_get_indices(val_MDG, global_node_to_index)

# Convert to PyTorch Geometric Data
train_data = from_networkx(remapped_train_MDG)
train_data.x = train_indices

val_data = from_networkx(remapped_val_MDG)
val_data.x = val_indices

train_data.x = train_data.x.long()  # Convert to LongTensor
val_data.x = val_data.x.long()  # Convert to LongTensor

In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv, GCNConv
from torch_geometric.nn.models import VGAE

class GATGCNVarEncoder(torch.nn.Module):
    def __init__(self, max_nodes, embedding_dim, out_channels):
        super(GATGCNVarEncoder, self).__init__()
        self.node_emb = torch.nn.Embedding(max_nodes, embedding_dim)
        self.conv1 = GCNConv(embedding_dim, 2 * out_channels)

        # Two parallel layers for mean and log-variance
        self.conv_mu = GATConv(2 * out_channels, out_channels, heads=1, dropout=0.2)
        self.conv_logvar = GATConv(2 * out_channels, out_channels, heads=1, dropout=0.2)

    def forward(self, x, edge_index):
        x = self.node_emb(x)
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.2, training=self.training)
        
        # Separate computation for mean and log-variance
        mu = self.conv_mu(x, edge_index)
        logvar = self.conv_logvar(x, edge_index)
        return mu, logvar

# Usage
max_nodes = 5000  # Set to a number higher than your expected number of nodes
embedding_dim = 16
out_channels = 16

var_encoder = GATGCNVarEncoder(max_nodes, embedding_dim, out_channels)
# Create the VGAE model
model = VGAE(var_encoder)

In [5]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import umap.umap_ as umap
import numpy as np

def validate_vgae(model, val_data):
    model.eval()
    with torch.no_grad():
        # Encode the validation data
        z_val = model.encode(val_data.x, val_data.edge_index)

        # Access mu and logstd directly
        mu_val, logstd_val = model.__mu__, model.__logstd__

        # Calculate the reconstruction loss
        val_recon_loss = model.recon_loss(z_val, val_data.edge_index)

        # Calculate the KL divergence
        val_kl_loss = model.kl_loss(mu_val, logstd_val)

        # Combine the losses
        val_loss = val_recon_loss + (1 / val_data.num_nodes) * val_kl_loss

    return val_loss.item()



def broad_search(embeddings, step, max_clusters):
    best_score = -1
    best_n_clusters = 0
    for n_clusters in range(2, max_clusters + 1, step):
        score = calculate_silhouette_score(embeddings, n_clusters)
        if score > best_score:
            best_score = score
            best_n_clusters = n_clusters
    return best_score, best_n_clusters

def detailed_search(embeddings, start, end, step):
    best_score = -1
    best_n_clusters = 0
    for n_clusters in range(start, end + 1, step):
        score = calculate_silhouette_score(embeddings, n_clusters)
        if score > best_score:
            best_score = score
            best_n_clusters = n_clusters
    return best_score, best_n_clusters

def calculate_silhouette_score(embeddings, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
    cluster_labels = kmeans.fit_predict(embeddings)
    return silhouette_score(embeddings, cluster_labels)

c:\Users\ycy6y\.conda\envs\pathway_siamese_network\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\ycy6y\.conda\envs\pathway_siamese_network\Lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\ycy6y\.conda\envs\pathway_siamese_network\Lib\site-

In [6]:
from torch.optim import Adam

optimizer = Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

def train_vgae(model, data, optimizer):
    model.train()
    optimizer.zero_grad()

    # Encode the training data
    z = model.encode(data.x, data.edge_index)

    # Access mu and logstd directly
    mu, logstd = model.__mu__, model.__logstd__

    # Calculate the reconstruction loss
    recon_loss = model.recon_loss(z, data.edge_index)

    # Calculate the KL divergence
    kl_loss = model.kl_loss(mu, logstd)

    # Combine the losses
    total_loss = recon_loss + (1 / data.num_nodes) * kl_loss

    # Backpropagation and optimization
    total_loss.backward()
    optimizer.step()

    return total_loss.item()



In [7]:
import torch
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Early stopping and model saving parameters
patience = 10
best_val_score = -1
epochs_no_improve = 0
early_stop = False
best_model_state = None  # To store the best model state

# Scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

for epoch in range(200):
    loss = train_vgae(model, train_data, optimizer)  # Updated train function call
    val_loss = validate_vgae(model, val_data)  # Updated validate function call

    print(f'Epoch: {epoch + 1}, Loss: {loss:.4f}, Val Loss: {val_loss:.4f}')

    scheduler.step(val_loss)  # Update based on val_loss

    # Early stopping based on the validation loss
    if val_loss < best_val_score or best_val_score == -1:
        best_val_score = val_loss
        epochs_no_improve = 0
        best_model_state = model.state_dict()
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print("Early stopping triggered")
        break

if best_model_state is not None:
    torch.save(best_model_state, 'best_vgae_model.pth')  # Update the file name
    print("Best model saved.")
else:
    print("No model improvement was observed.")

Epoch: 1, Loss: 5.2999, Val Loss: 2.4055
Epoch: 2, Loss: 4.1038, Val Loss: 1.9676
Epoch: 3, Loss: 3.3534, Val Loss: 1.7506
Epoch: 4, Loss: 2.9585, Val Loss: 1.6269
Epoch: 5, Loss: 2.5971, Val Loss: 1.5697
Epoch: 6, Loss: 2.4067, Val Loss: 1.5289
Epoch: 7, Loss: 2.1629, Val Loss: 1.4990
Epoch: 8, Loss: 2.0050, Val Loss: 1.4561
Epoch: 9, Loss: 1.8515, Val Loss: 1.4594
Epoch: 10, Loss: 1.7590, Val Loss: 1.4412
Epoch: 11, Loss: 1.7214, Val Loss: 1.4480
Epoch: 12, Loss: 1.6187, Val Loss: 1.4449
Epoch: 13, Loss: 1.5593, Val Loss: 1.4341
Epoch: 14, Loss: 1.5325, Val Loss: 1.4365
Epoch: 15, Loss: 1.4885, Val Loss: 1.4298
Epoch: 16, Loss: 1.4440, Val Loss: 1.4106
Epoch: 17, Loss: 1.4118, Val Loss: 1.4038
Epoch: 18, Loss: 1.3883, Val Loss: 1.3889
Epoch: 19, Loss: 1.3499, Val Loss: 1.3879
Epoch: 20, Loss: 1.3402, Val Loss: 1.3560
Epoch: 21, Loss: 1.3081, Val Loss: 1.3540
Epoch: 22, Loss: 1.2857, Val Loss: 1.3475
Epoch: 23, Loss: 1.2840, Val Loss: 1.3500
Epoch: 24, Loss: 1.2793, Val Loss: 1.3476
E

In [9]:
# Step 1: Load the trained model and generate embeddings
model.load_state_dict(torch.load('vgae_model.pth'))
model.eval()

with torch.no_grad():
    z = model.encode(train_data.x, train_data.edge_index)
    embeddings = z.cpu().numpy()

# Step 2: Apply Spectral Clustering
from sklearn.cluster import SpectralClustering

n_clusters = 92
spectral_clustering = SpectralClustering(n_clusters=n_clusters, random_state=42, affinity='nearest_neighbors')
cluster_labels = spectral_clustering.fit_predict(embeddings)

# Step 3: Map Clusters to Gene Names
index_to_gene = {index: gene for gene, index in global_node_to_index.items()}
gene_names = [index_to_gene[i] for i in range(len(embeddings))]
gene_cluster_pairs = list(zip(gene_names, cluster_labels))

# Step 4: Write Results to CSV
import csv

with open('vgae_spectral_gene_cluster_assignments.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Gene', 'Cluster'])
    for gene, cluster in gene_cluster_pairs:
        writer.writerow([gene, cluster])

c:\Users\ycy6y\.conda\envs\pathway_siamese_network\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [10]:
# Assuming the rest of your setup is the same

# Step 2: Apply KMeans Clustering instead of Spectral Clustering
from sklearn.cluster import KMeans

n_clusters = 92  # The number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(embeddings)

# Mapping Clusters to Gene Names
index_to_gene = {index: gene for gene, index in global_node_to_index.items()}
gene_names = [index_to_gene[i] for i in range(len(embeddings))]
gene_cluster_pairs = list(zip(gene_names, cluster_labels))

# Write Results to CSV
import csv

with open('vgae_kmeans_gene_cluster_assignments.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Gene', 'Cluster'])
    for gene, cluster in gene_cluster_pairs:
        writer.writerow([gene, cluster])

c:\Users\ycy6y\.conda\envs\pathway_siamese_network\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [12]:
from skfuzzy.cluster import cmeans

# Convert embeddings to the required format for FCM
embeddings_T = np.transpose(embeddings)  # Transpose embeddings for cmeans

# Apply Fuzzy C-Means
cntr, u, _, _, _, _, _ = cmeans(embeddings_T, n_clusters, 2, error=0.005, maxiter=1000, init=None)

# u contains the membership matrix
# Transpose the matrix so each row corresponds to a gene
membership_matrix = np.transpose(u)

# Writing Fuzzy Clustering Results to CSV
with open('vgae_fcm_gene_cluster_assignments.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    header = ['Gene'] + [f'Cluster_{i}' for i in range(n_clusters)]
    writer.writerow(header)

    for gene_name, memberships in zip(gene_names, membership_matrix):
        writer.writerow([gene_name] + list(memberships))